# Outline

Why this tutorial will be useful:
- This tutorial builds on previous exercises to expand your data science toolkit
- You will learn a handful of key techniques for visualizing high-dimensional data
- <font color='green'>As previously, you will solely need to change the code at positions marked with ```# <- adapt code here```</font>, however you are encouraged to experiment further

Main purpose of tutorial:
- Give exposure to popular techniques for qualitatively presenting relationships in high-dimensional datasets
- Provide visualization techniques that will be valuable in your own analyses

Additional purposes of tutorial:
- Give an overview of Principal Components Analysis (PCA), nonlinear correlation, and hierarchical clustering
- Learn to identify outlier samples in high-dimensional data

Stages:
- Stage 0: Load necessary components
- Stage 1: PCA (and plotting)
- Stage 2: Plotting Heatmaps

# Stage 0: Load necessary components

Refer to "02_homework" for a review of the basics of data and package import

In [ ]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
import numpy as np

In [ ]:
# import expression data
counts = pd.read_csv("/Users/rogangrant/Box/chaperome_student_course/chaperome_class/homework/Human.RPKM.txt",
                    sep = " ") # <- adapt code here

You will note that the gene identifiers are in ensembl format. The benefit of this is that these gene IDs will **never** change, whereas common gene names are highly variable and follow no clear convention. For the purposes of presenting data, however, most will prefer common gene names. For this exercise, we will substitute them.

In [ ]:
# move ensembl IDs into a column for merging
counts = counts.reset_index()
counts.head()

In [ ]:
# Conversion
# import conversion table
gene_conversions = pd.read_csv("/Users/rogangrant/Box/chaperome_student_course/chaperome_class/homework/human_gene_name_conversions.csv") # <- adapt code here
name_conversions = gene_conversions[["ensembl_gene_id", "external_gene_name"]]
name_conversions.head()

In [ ]:
# perform merge
counts = pd.merge(counts,
                 name_conversions,
                 left_on = "index",
                 right_on = "ensembl_gene_id",
                 how = "inner") #take intersection of datasets
counts.head()

In [ ]:
#remove unnecessary columns
counts = counts.set_index("external_gene_name")
counts = counts.drop(["index", "ensembl_gene_id"], axis = 1) #axis = 1 selects columns
counts.head()

# Stage 1: PCA (and plotting)

Now that we have our data in a workable format, we can perform principal components analysis. Volumes have been written on this technique, and with good reason; it is extremely powerful for handling high-dimensional data. Back in the days of qPCR, experiments would generate expression data for just a few genes. If you want to compare patterns of expression for a group of samples for just two genes, the procedure is very simple: plot expression of gene A on the X-axis, and gene B on the Y-axis for each sample. A third gene could even go on the Z-axis for a 3D plot. RNA-seq, on the other hand, generates data for every transcript in the human genome (43256, in this case). Plotting this number of dimensions is effectively impossible in our 3-dimensional universe. 

How do we get past this? One option is to plot each dimension (gene) individually, but this is fairly tedious. We will see how this is made more possible with heatmaps later. Alternatively, we can reduce the dimensionality of our data. While this clearly has the risk of reducing information value (and certainly does, to some degree), this turns out to be an excellent way to compare high-dimensional datasets in a semi-quantitative way. Without going too far into the linear algebra, PCA identifies features (genes, in this case) which are highly correlated, and groups them together into a single "principal component". This is often referred to an eigenvector, which is the linear combination of these features into a single "eigenfeature". PCA then iteratively generates additional principal components from groups of features that are correlated, but orthogonal (not correlated) to existing principal components. Convention is generally to generate 2-3 principal components.

This works particularly well with gene expression data, because groups of genes are often co-regulated. By examining the contribution of each gene to each eigenvector (or "eigengene", if you will), one can even begin the infer the biological significance. Most simply, however, PCA offers an excellent way to roughly compare the relationship between datasets for a given experiment. By examining how the samples cluster together on a PCA plot, we can better understand which samples behave similarly.

Let's try this with the complete human dataset.

## Stage 1.1: pre-processing and PCA calculation

To perform PCA, we need to manipulate the expression matrix in the following ways:
1. Transpose the dataset, or flip it so that genes are columns, and samples are rows
2. Scale expression values so that they vary from 0 to 1 and contribute equally to the PCA
3. Center expression values about 0 (this is performed implicitly)

In [ ]:
#transpose
counts_for_pca = counts.transpose()
counts_for_pca = counts_for_pca.rename_axis(["sample"])
counts_for_pca.head()

In [ ]:
#scale from 0 to 1
counts_for_pca_scaled = StandardScaler().fit_transform(counts_for_pca)

# center and perform PCA
pca = PCA(n_components=3) # we will do 3 PCs
PCs = pca.fit_transform(counts_for_pca_scaled)
pca_dataframe = pd.DataFrame(data = PCs,
                           columns = ['PC1', 'PC2', 'PC3'])
pca_dataframe.head()

## Stage 1.2: visualization 

Now that we have our PCA data, let's add our metadata and plot it

In [ ]:
#add metadata

pca_dataframe["sample"] = counts_for_pca.index
#make sample names match larger dataset (which includes a species prefix)
for i in range(pca_dataframe["sample"].size):
    pca_dataframe["sample"][i] = "Human." + pca_dataframe["sample"][i]

md = pd.read_csv("/Users/rogangrant/Box/chaperome_student_course/chaperome_class/homework/Kaessman_sample_metadata.csv") # <- adapt code here
pca_dataframe = pd.merge(pca_dataframe,
                        md,
                        on = "sample",
                        how = "inner")
pca_dataframe.head()

In [ ]:
# plot scatter
sns.scatterplot(data = pca_dataframe,
                x = "PC1",
                y = "PC2",
               hue = "tissue")

In [ ]:
sns.scatterplot(data = pca_dataframe,
                x = "PC2",
                y = "PC3",
               hue = "tissue")

As expected, samples corresponding to a single tissue cluster together, as do similar tissues. It does appear that some testis samples may have been mishandled or differently processed in some way, as evidenced by their strange behavior on PC1. For more interesting visualization, try this for a tissue of interest for all organisms in the dataset. The trends are quite revealing.

You can also try changing which factors you map to "hue", or you can include a "style" aesthetic, which will map a factor to the shape of a point. Try using "age" as a mapping.

# Stage 2: Plotting heatmaps

In the simplest sense, heatmaps are just visualizations of a matrix. The rows and columns are the same, and the color or intensity of a given sample corresponds to its value. In general, brighter colors correspond to higher values, whereas "cooler" colors correspond to lower values. 

It is quite common to "cluster" rows and/or columns, such that more similar samples are organized together; this allows for better pattern recognition. While many clustering methods exist, they are all similar in principle. First, pairwise "distances" are calculated for each sample. This can be anything from correlation coefficients, to average differences between measurements, and depends largely on the type of data and the comparison. In all cases, the most similar datasets are identified iteratively, and joined in reverse, stepwise order until all datasets have been joined. This then gives an approximation of dataset similarity. By sorting the samples in order of similarity, much more obvious patterns tend to emerge. We will see this firsthand.

## Stage 2.1: average expression values by condition

As with any good biological dataset, we have many replicates per sample. In order to reduce the complexity of our plot, we will average expression for each condition. As before, consider what impact a mean may have on these representations, as compared to median or other statistics.

To do this, we will have to derive a new matrix from our raw data, with just one column per sample. Have a look at the old matrix as necessary to get an idea of how this will work.

In [ ]:
#first, make the new column names, which will be the unique values after stripping replicate number
mean_cols = list(counts.columns)
regex = re.compile('\.\d+$') #i.e. ".[one or more digits][end of string]"
for i in range(len(mean_cols)):
    mean_cols[i] = mean_cols[i][0:regex.search(mean_cols[i]).start()] #remove the regex shown above
mean_cols = np.unique(mean_cols)
mean_cols

In [ ]:
#now create an empty data frame that we will fill with mean values
mean_counts = pd.DataFrame(columns = mean_cols, index = counts.index)
mean_counts.head()

In [ ]:
# now we will iterate through each condition and get the mean expression value for each gene
for condition in mean_cols:
    #first, get the column names of all replicates for a given condition
    replicates = []
    for column in counts.columns:
        if column.startswith(condition):
            replicates.append(column)
    
    #then create a subset of counts with just those columns
    subset = counts[replicates]
    
    #now take the mean value of every row
    column_means = subset.apply(
    np.mean, 
    axis='columns')
    
    #and place these means in our means dataframe
    mean_counts[condition] = column_means
    
mean_counts.head()

Now we have a matrix of mean expression values. You can adapt this code as you please to summarize your own subsets. Try performing a PCA on this dataset if you have time. Do the mean values recapitulate the patterns you observed above?

## Stage 2.2: Plot heatmap

This section should be fairly straightforward. One important thing to consider, however, is how you will scale your values (if at all). Because some genes are expressed at very high levels, data for low-expression genes will likely be difficult to interpret due to the wide range of the color scale. Similar issues may also occur at the higher end.

Some common solutions to this problem:
1. Log2-transform all of the values
2. Plot z-scores for each gene $Z_i = \frac{ExpressionValue_i_ - Mean_gene.x_}{SD_gene.x_}$

We will leave it up to you to try these operations and see what best captures the trends in your data. For now, we will just plot raw values, and we will do it without clustering.

Note: we will just do this for the first 100 genes, but you will need to perform your own subsetting.

In [ ]:
plt.rcParams["figure.figsize"] = (20,25) #change this to accommodate large plot
example_subset = mean_counts.iloc[0:99,:]
sns.heatmap(data = example_subset, 
            robust = True) #robust sets the color scale to more realistic values for visualization

This actually worked fairly well, so we can proceed without any transformations (at least for the sake of this example). You are encouraged to try to find the best normalization technique for your dataset. Now, how will this plot change if we cluster?

In [ ]:
sns.clustermap(data = example_subset, 
               method="ward",
               robust = True,
              figsize=[20,25])

In this case, we have used Ward's Method for clustering, which is a popular method in the data science field. In the simplest sense, this groups samples with a minimal pairwise variance. Wikipedia is a great resource for further reading on Ward's Method, and hierchical clustering, in general.

What we learn from this plot, as opposed to the unclustered plot above, is that our samples group very well by tissue, as would be expected. This is impressive, given that small number of genes included. It also nicely complements the results from the PCA, such as the significant overlap between brain (cerebrum) and cerebellum.

We can also observe interesting gene groupings, which may reveal potential regulons (groups of co-expressed genes). Many "housekeeping" genes are observed in the high-expression group at the bottom, whereas tightly regulated genes such as HOXA11 fall into the "dark" cluster. See what trends you can identify.

# Conclusion

Hopefully this tutorial has served as a nice complement to homework 2, and has given some ideas of how to best represent high-dimensional data. This is just a short overview of the methods available, but PCA and cluster-/heat-mapping will almost always be included in transcriptomic data analysis. For higher level analysis, new multi-dimensional scaling and plotting methods are emerging, such as T-SNE and UMAP. Machine learning is also becoming increasingly popular for pattern identification in high-dimensional data. These are all outside of the scope of this module, but interested students are encouraged to read further.